In [1]:
# All chemicals as candidate drugs
import pandas as pd
import numpy as np

# general drug
DDinter_df = pd.read_csv("D:/projects/MBKG/data/结构化&半结构化/DDinter/DDinter_triplets.csv", sep='\t', header=None)
DrugBank_df = pd.read_csv("D:/projects/MBKG/data/结构化&半结构化/DrugBank/DrugBank_triplets.csv", sep='\t', header=None)
GNBR_df1 = pd.read_csv("D:/projects/MBKG/data/结构化&半结构化/GNBR/chemical_disease_triplets2.csv", sep='\t', header=None)
GNBR_df2 = pd.read_csv("D:/projects/MBKG/data/结构化&半结构化/GNBR/chemical_gene_triplets2.csv", sep='\t', header=None)
STITCH_df = pd.read_csv("D:/projects/MBKG/data/结构化&半结构化/STITCH/STITCH_triplets2.csv", sep='\t', header=None)
FreeText_df1 = pd.read_csv("D:/projects/MBKG/data/FreeText/chemical_disease_triplets.csv", sep='\t', header=None)
FreeText_df2 = pd.read_csv("D:/projects/MBKG/data/FreeText/chemical_gene_triplets.csv", sep='\t', header=None)
general_drug_df = pd.concat([DDinter_df[0], DDinter_df[2], DrugBank_df[0], GNBR_df1[0], GNBR_df2[0], STITCH_df[0], FreeText_df1[0], FreeText_df2[0]])
general_drug_df.drop_duplicates(inplace=True)  # 36133
general_drug_df.rename('entity', inplace=True)
general_drug_list = general_drug_df.values.tolist()  # 36133

# traditional chinese medicine
TCMID_df = pd.read_csv("D:/projects/MBKG/data/结构化&半结构化/TCMID/TCMID_triplets.csv", sep='\t', header=None)
tcm_drug_df = TCMID_df[0]
tcm_drug_df.drop_duplicates(inplace=True)  # 3004
tcm_drug_df.rename('entity', inplace=True)
tcm_drug_list = tcm_drug_df.values.tolist()  # 3004

In [2]:
# Extract all genes possibly associated with bone loss from literature related to both microgravity and bone loss
genes = pd.read_csv('mg_bone_loss_related_genes.csv', sep='\t', header=None)
genes.drop_duplicates(inplace=True)
genes_list = genes.values[:, 0].tolist()
print(len(genes_list))

94


In [3]:
treatment = ['chemical:gene:A+', 'chemical:gene:E+']

In [4]:
# Load entity & relation in pretrained model
entitiy_df = pd.read_table("D:/projects/MBKG/code/Train/TransE_l2/entities.tsv", sep='\t', header=None, names=['id', 'entity'])
relation_df = pd.read_table("D:/projects/MBKG/code/Train/TransE_l2/relations.tsv", sep='\t', header=None, names=['id', 'rel'])

In [5]:
# ID mapping
general_drug_df = pd.merge(general_drug_df, entitiy_df, on='entity')
tcm_drug_df = pd.merge(tcm_drug_df, entitiy_df, on='entity')

treatment_df = pd.merge(pd.DataFrame(treatment, columns=['rel']), relation_df)

genes_df = pd.merge(pd.DataFrame(genes_list, columns=['entity']), entitiy_df)

In [52]:
genes_df

,entity,id
0,ENTREZ:81668,39646
1,ENTREZ:3486,3395
2,ENTREZ:632,3371
3,ENTREZ:25295,44851
4,ENTREZ:5741,38120
...,...,...
88,ENTREZ:66026,38101
89,ENTREZ:18477,40017
90,ENTREZ:109648,42120
91,ENTREZ:21823,39709


In [8]:
# Load embeddings
import torch
import numpy as np

entity_emb = np.load("D:/projects/MBKG/code/ckpts/TransE_l2_MGKG_0/MGKG_TransE_l2_entity.npy")
rel_emb = np.load("D:/projects/MBKG/code/ckpts/TransE_l2_MGKG_0/MGKG_TransE_l2_relation.npy")

general_drug_ids = torch.tensor(general_drug_df['id'].values.tolist())
tcm_drug_ids = torch.tensor(tcm_drug_df['id'].values.tolist())
treatment_ids = torch.tensor(treatment_df['id'].values.tolist())
genes_ids = torch.tensor(genes_df['id'].values.tolist())

general_drug_embs = torch.tensor(entity_emb[general_drug_ids])
tcm_drug_embs = torch.tensor(entity_emb[tcm_drug_ids])
treatment_embs = [torch.tensor(rel_emb[rid]) for rid in treatment_ids]

## Drug Repurposing Based on Edge Score
We use following algorithm to calculate the edge score. Note, here we use logsigmiod to make all scores < 0. The larger the score is, the stronger the $h$ will have $r$ with $t$.

$\mathbf{d} = \gamma - ||\mathbf{h}+\mathbf{r}-\mathbf{t}||_{2}$

$\mathbf{score} = \log\left(\frac{1}{1+\exp(\mathbf{-d})}\right)$

When doing drug repurposing, we only use the treatment related relations.

In [15]:
import torch.nn.functional as F

gamma = 12.0  # 模型训练时使用的γ值


def transE_l2(head, rel, tail):
    score = head + rel - tail
    return gamma - torch.norm(score, p=2, dim=-1)

### western medicine

In [48]:
# general_drug 大部分都是西药，可能包含一些中药成分
general_drugs_per_gene = []
general_scores_per_gene = []

for gene_id in genes_ids:
    gene_emb = entity_emb[gene_id]
    drugs_per_treat = []
    scores_per_treat = []
    for rid in range(len(treatment_embs)):
        treatment_emb = treatment_embs[rid]
        score = F.logsigmoid(transE_l2(general_drug_embs, treatment_emb, gene_emb))
        scores_per_treat.append(score)
        drugs_per_treat.append(general_drug_ids)
    general_drugs_per_gene.append(torch.cat(drugs_per_treat))
    general_scores_per_gene.append(torch.cat(scores_per_treat))

In [86]:
# sort scores in decending order
for i in range(len(general_scores_per_gene)):
    scores = general_scores_per_gene[i]
    drugs = general_drugs_per_gene[i]
    scores = scores.clone().detach()
    drugs_idx = torch.argsort(scores, descending=True)
    sort_scores = scores[drugs_idx].numpy()
    sort_drugs = drugs[drugs_idx].numpy()
    gene = genes_df.iloc[i, 0]

    # select top 100
    topk=100

    _, general_drugs_unique_idx = np.unique(sort_drugs, return_index=True)
    general_drugs_topk_idx = np.sort(general_drugs_unique_idx)[:topk]
    proposed_general_drugs_ids = sort_drugs[general_drugs_topk_idx]
    proposed_general_drugs = []
    for id in proposed_general_drugs_ids:
        proposed_general_drugs.append(general_drug_df.loc[general_drug_df['id'] == id, 'entity'].values[0])
    proposed_general_drugs_scores = sort_scores[general_drugs_topk_idx]
    with open('modern_medicine_via_genes.csv', 'a+') as f:
        f.write(gene + '\n')
        for d, s in zip(proposed_general_drugs, proposed_general_drugs_scores):
            f.write(f"{d}\t{s}\n")
        f.write('\n')

### herb

In [87]:
tcm_drugs_per_gene = []
tcm_scores_per_gene = []

for gene_id in genes_ids:
    gene_emb = entity_emb[gene_id]
    drugs_per_treat = []
    scores_per_treat = []
    for rid in range(len(treatment_embs)):
        treatment_emb = treatment_embs[rid]
        score = F.logsigmoid(transE_l2(tcm_drug_embs, treatment_emb, gene_emb))
        scores_per_treat.append(score)
        drugs_per_treat.append(tcm_drug_ids)
    tcm_drugs_per_gene.append(torch.cat(drugs_per_treat))
    tcm_scores_per_gene.append(torch.cat(scores_per_treat))

# sort scores in decending order
for i in range(len(tcm_scores_per_gene)):
    scores = tcm_scores_per_gene[i]
    drugs = tcm_drugs_per_gene[i]
    scores = scores.clone().detach()
    drugs_idx = torch.argsort(scores, descending=True)
    sort_scores = scores[drugs_idx].numpy()
    sort_drugs = drugs[drugs_idx].numpy()
    gene = genes_df.iloc[i, 0]

    # select top 100
    topk=100

    _, tcm_drugs_unique_idx = np.unique(sort_drugs, return_index=True)
    tcm_drugs_topk_idx = np.sort(tcm_drugs_unique_idx)[:topk]
    proposed_tcm_drugs_ids = sort_drugs[tcm_drugs_topk_idx]
    proposed_tcm_drugs = []
    for id in proposed_tcm_drugs_ids:
        proposed_tcm_drugs.append(tcm_drug_df.loc[tcm_drug_df['id'] == id, 'entity'].values[0])
    proposed_tcm_drugs_scores = sort_scores[tcm_drugs_topk_idx]
    with open('traditional_Chinese_medicine_via_genes.csv', 'a+') as f:
        f.write(gene + '\n')
        for d, s in zip(proposed_tcm_drugs, proposed_tcm_drugs_scores):
            f.write(f"{d}\t{s}\n")
        f.write('\n')